In [224]:
from instagram_web_api import Client, ClientCompatPatch, ClientError, ClientLoginError
from datetime import datetime

import hashlib
import string
import random
import time
import json
import os.path
import logging
import argparse

# просто Client не работает, поэтому переопределена функция _extract_rhx_gis для захода в web инста
class MyClient(Client):
    @staticmethod
    def _extract_rhx_gis(html):
        options = string.ascii_lowercase + string.digits
        text = ''.join([random.choice(options) for _ in range(8)])
        return hashlib.md5(text.encode())

def getWebAPI(username, password):
    return MyClient(auto_patch = True, authenticate = True, username = username, password = password)

In [225]:
class Comment:
    
    # просто структурка для коммента, в которой хранятся автор, время и текст коммента
    
    def __init__(self, com_author, com_time, com_text):
        self.com_author = com_author
        self.com_time = com_time
        self.com_text = com_text
    
def getUserID(api, username):
    return api.user_info2(username)['id']
    
# возвращает список всех комментов к посту
def getPostComments(api, media_id):
    res = []
    max_id = ' '
    next_max_id = ''
    
    while (max_id != next_max_id):
        
        time.sleep(3) # ждем некоторое время, так как нельзя подряд делать бесконечное число http-запросов
        
        max_id = next_max_id
        post_comments = authed_web_api.media_comments(media_id, count = 50, end_cursor = max_id) # загружаем комменты
        
        # если комменты кончились, ливаем
        if (len(post_comments) == 0):
            break
        
        # смотрим, на каком комменте закончилась выгрузка
        # реверсим массив комментов, чтобы итоговый список получился в убывающем порядке
        next_max_id = post_comments[0]['id']
        post_comments.reverse()
        
        # оставляем только нужную информацию
        for comm in post_comments:
            res.append(Comment(comm['owner']['username'], comm['created_at'], comm['text']))
    
    return res

# возвращает media_id всех медиа профиля
def getAllProfileMediaList(api, username):
    user_id = getUserID(api, username)
    
    media_list = []
    next_page = ''
    has_next_page = True
    
    while (has_next_page):
        
        time.sleep(3) # ждем некоторое время, так как нельзя подряд делать бесконечное число http-запросов
        
        all_media = api.user_feed(user_id, count = 50, end_cursor = next_page, extract = False)
        
        # записываем id полученных медиа профиля в лист
        for item in all_media['data']['user']['edge_owner_to_timeline_media']['edges']:
            media_list.append(item['node']['shortcode'])
        
        # смотрим, есть ли следующая страница
        has_next_page = all_media['data']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
        
        # присваиваем ссылку на следующую страницу
        next_page = all_media['data']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        
    return media_list

# возвращает список всех комментов к профилю
def getAllProfileComments(api, username):
    
    # достаем все media_id
    media_list = getAllProfileMediaList(api, username)
    
    # получаем все комментарии к профилю
    res = []
    for media_id in media_list:
        res.extend(getPostComments(api, media_id))
    
    return res

In [226]:
api = getWebAPI('testinstapp', 'testinstapp1')

username = 'mr_justadog'

comments = getAllProfileComments(api, username)
for comm in comments:
    print('author: ' + comm.com_author + ' time: ' + str(datetime.fromtimestamp(comm.com_time)) + ' text: ' + comm.com_text)

author: mr_justadog time: 2020-01-27 21:09:46 text: @holymosh ты просто первую полностью не видел)0
author: holymosh time: 2020-01-27 21:09:03 text: А тебе на второй фотке ещё налили?)
author: mr_justadog time: 2019-12-06 15:24:36 text: @permyakovaart не выдержала психологического давления
author: _nastasjushka time: 2019-12-06 14:59:33 text: Алло, сегодня опять блистаю🤩
author: permyakovaart time: 2019-12-06 14:45:37 text: А где мотимотичька?
author: eugeneblauberg time: 2019-12-05 22:10:49 text: @erzhenavamp еее
author: erzhenavamp time: 2019-12-05 21:32:55 text: подпись кайф
author: mr_justadog time: 2019-11-03 23:36:27 text: @permyakovaart да говорю же, мотемотичька зверствует
author: permyakovaart time: 2019-11-03 22:58:05 text: Да на что там опять все смотрят?!
author: vy4c time: 2019-11-02 23:32:54 text: Офигенный костюм, мне очень зашёл)
author: permyakovaart time: 2019-10-28 23:34:06 text: @anasta_su ахуена
author: anasta_su time: 2019-10-28 23:33:38 text: @permyakovaart ЧЕТВЕ